<div style="display: flex; align-items: left;">
    <a href="https://sites.google.com/corp/google.com/genai-solutions/home?authuser=0">
        <img src="https://storage.googleapis.com/miscfilespublic/Linkedin%20Banner%20%E2%80%93%202.png" style="margin-right">
    </a>
</div>

In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


<h1 align="center">Open Data QnA - Chat with your SQL Database</h1> 

---

This notebook first walks through the Vector Store Setup needed for running the Open Data QnA application. 

Currently supported Source DBs are: 
- PostgreSQL on Google Cloud SQL 
- BigQuery

Furthermore, the following vector stores are supported 
- pgvector on PostgreSQL 
- BigQuery vector


The setup part covers the following steps: 
> 1. Configuration: Intial GCP project, IAM permissions, Environment  and Databases setup including logging on Bigquery for analytics

> 2. Creation of Table, Column and Known Good Query Embeddings in the Vector Store  for Retreival Augmented Generation(RAG)


Afterwards, you will be able to run the Open Data QnA Pipeline to generate SQL queries and answer questions over your data source. 

The pipeline run covers the following steps: 

> 1. Take user question and generate sql in the dialect corresponding to data source

> 2. Execute the sql query and retreive the data

> 3. Generate natural language respose and charts to display

> 4. Clean Up resources



### 📒 Using this interactive notebook

If you have not used this IDE with jupyter notebooks it will ask for installing Python + Jupyter extensions. Please go ahead install them

Click the **run** icons ▶️  of each cell within this notebook.

> 💡 Alternatively, you can run the currently selected cell with `Ctrl + Enter` (or `⌘ + Enter` on a Mac).

> ⚠️ **To avoid any errors**, wait for each section to finish in their order before clicking the next “run” icon.

This sample must be connected to a **Google Cloud project**, but nothing else is needed other than your Google Cloud project.

You can use an existing project. Alternatively, you can create a new Cloud project [with cloud credits for free.](https://cloud.google.com/free/docs/gcp-free-tier)

# 🚧 **0. Prerequisites**

Make sure that Google Cloud CLI is installed before moving to the next cell! You can refer to the link below for guidance

Installation Guide: https://cloud.google.com/sdk/docs/install

##  **0.1. Setup Poetry Environment and Setup GCP Project** 

### 💻 **Install Code Dependencies (Create and setup venv)**
Install the dependencies by runnign the poetry commands below 

Note: Below command runs with default Python Kernel and we will change that to Kernel from venv after this execution below

In [1]:
# Install poetry
! pip uninstall poetry -y
! pip install poetry --quiet

#Run the poetry commands below to set up the environment
!poetry lock #resolve dependecies (also auto create poetry venv if not exists)
!poetry install --quiet #installs dependencies
!poetry env info #Displays the evn just created and the path to it

Found existing installation: poetry 1.8.3
Uninstalling poetry-1.8.3:
  Successfully uninstalled poetry-1.8.3
Creating virtualenv talktodata in /home/admin_/Talk2Data/.venv
Updating dependencies
Resolving dependencies... (42.7s)Resolving dependencies... (0.9s)Resolving dependencies... (6.1s)Resolving dependencies... (8.2s)Resolving dependencies... (13.6s)Resolving dependencies... (16.9s)Resolving dependencies... (26.6s)Resolving dependencies... (26.7s)

Writing lock file

Virtualenv
Python:         3.10.12
Implementation: CPython
Path:           /home/admin_/Talk2Data/.venv
Executable:     /home/admin_/Talk2Data/.venv/bin/python
Valid:          True

Base
Platform:   linux
OS:         posix
Python:     3.10.12
Path:       /usr
Executable: /usr/bin/python3.10


### 📌 **Important Step: Activate your virtual environment and authenticate with Google Cloud CLI [Run all these on Terminal]**

Once the venv created either in the local directory or in the cache directory. Open the terminal on the same machine where your notebooks are running and start running the below commands.


```
poetry shell #this command should activate your venv and you should see it enters into the venv

##inside the activated venv shell

gcloud auth login
gcloud auth application-default login
gcloud services enable \
    serviceusage.googleapis.com \
    cloudresourcemanager.googleapis.com --project <<Enter Project Id>>
gcloud auth application-default set-quota-project <<Enter Project Id for using resources>>

```
For IDEs adding Juypter Extensions will automatically give you option to change the kernel. If not, manually select the python interpreter in your IDE (The exact is shown in the above cell. Path would look like e.g. /home/admin_/Talk2Data/.venv/bin/python or ~cache/user/Talk2Data/.venv/bin/python)

**Extra Steps if you are running inside Jupyter Lab or Jupyter Environments on Workbench etc**

We need to manually add venv as Kernel in the those instance where you don't have choice to select the path manually like above.

Run the steps above and continue with below

```
##still inside the activated venv shell

pip install jupyter

ipython kernel install --name "openqna-venv" --user 

```

Restart your kernel or close the exsiting notebook and open again, you should now see the "openqna-venv" in the kernel drop down

**What did we do here?**

* Created Application Default Credentials to use for the code
* Added venv to kernel to select for runningt the notebooks (For standalone Jupyter setups like Workbench etc)

### Set Python Module Path to Root

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

### 🔗 **Connect Your Google Cloud Project**
Time to connect your Google Cloud Project to this notebook. 

In [2]:
#@markdown Please fill in the value below with your GCP project ID and then run the cell.
PROJECT_ID = "my_project_id"

# Quick input validations.
assert PROJECT_ID, "⚠️ Please provide your Google Cloud Project ID"

# Configure gcloud.
!gcloud config set project {PROJECT_ID}
print(f'Project has been set to {PROJECT_ID}')

os.environ['GOOGLE_CLOUD_QUOTA_PROJECT']=PROJECT_ID
os.environ['GOOGLE_CLOUD_PROJECT']=PROJECT_ID

Updated property [core/project].
Project has been set to ten-p-o


### ⚙️ **Enable Required API Services in the GCP Project**

In [3]:
#Enable all the required APIs for the Open Data QnA solution

!gcloud services enable \
  cloudapis.googleapis.com \
  compute.googleapis.com \
  iam.googleapis.com \
  run.googleapis.com \
  sqladmin.googleapis.com \
  aiplatform.googleapis.com \
  bigquery.googleapis.com \

Operation "operations/acf.p2-340360744896-5c073402-7083-4bf9-8b4a-aa90101b4294" finished successfully.


# **1. Vector Store Setup** (Run once)
---

This section walks through the Vector Store Setup needed for running the Open Data QnA application. 

It covers the following steps: 
> 1. Configuration: Environment and Databases setup including logging on Bigquery for analytics

> 2. Creation of Table, Column and Known Good Query Embeddings in the Vector Store  for Retreival Augmented Generation(RAG)




## 📈 **1.1 Set Up your Data Source and Vector Store**

This section assumes that a datasource is already set up in your GCP project. If a datasource has not been set up, use the notebooks below to copy a public data set from BigQuery to Cloud SQL or BigQuery on your GCP project


Enabled Data Sources:
* PostgreSQL on Google Cloud SQL (Copy Sample Data: [0_CopyDataToCloudSqlPG.ipynb](0_CopyDataToCloudSqlPG.ipynb))
* BigQuery (Copy Sample Data: [0_CopyDataToBigQuery.ipynb](0_CopyDataToBigQuery.ipynb))

Enabled Vector Stores:
* pgvector on PostgreSQL 
* BigQuery vector


### 🤔 **Choose Data Source and Vector Store**

Fill out the parameters and configuration settings below. 
These are the parameters for connecting to the source databases and setting configurations for the vector store tables to be created. 
Additionally, you can specify whether you have and want to use known-good-queries for the pipeline run and whether you want to enable logging.

**Known good queries:** if you have known working user question <-> SQL query pairs, you can put them into the file `scripts/known_good_sql.csv`. This will be used as a caching layer and for in-context learning: If an exact match of the user question is found in the vector store, the pipeline will skip SQL Generation and output the cached SQL query. If the similarity score is between 90-100%, the known good queries will be used as few-shot examples by the SQL Generator Agent. 

**Logging:** you can enable logging. If enabled, a dataset is created in Big Query in your project, which will store the logging table and save information from the pipeline run in the logging table. This is especially helpful for debugging.

In [4]:
#[CONFIG]
embedding_model = 'vertex' # Options: 'vertex' or 'vertex-lang'
description_model = 'gemini-1.0-pro' # 'gemini-1.0-pro', 'gemini-1.5-pro', 'text-bison-32k'
data_source = 'bigquery' #  Options: 'bigquery' and 'cloudsql-pg' 
vector_store = 'bigquery-vector' # Options: 'bigquery-vector', 'cloudsql-pgvector'
logging = True # True or False 
kgq_examples = False # True or False 

#[GCP]
project_id = PROJECT_ID

#[PGCLOUDSQL]
# If you want to use PG as source, fill out the values below
pg_region = ''
pg_instance = ''
pg_database = ''
pg_user = ''
pg_password = ''
pg_schema = ''

#[BIGQUERY]
# If you want to use BQ as source, fill out the values below
bq_dataset_region = 'us-central1'
bq_dataset_name = 'imdb'

# Name for the BQ dataset created for bigquery-vector and/or logging
bq_opendataqna_dataset_name = 'opendataqna'
bq_log_table_name = 'audit_log_table' 
bq_table_list = None # either None or a list of table names in format ['reviews', 'ratings']

#Decode Region and Userdatabase based on source
if data_source == 'bigquery': dataset_region = bq_dataset_region; user_database=bq_dataset_name 
elif data_source == 'cloudsql-pg': dataset_region = pg_region; user_database=pg_schema

Quick input verifications below:

In [5]:

# Input verification - Source
assert data_source in {'bigquery', 'cloudsql-pg'}, "⚠️ Invalid DATA_SOURCE. Must be 'bigquery' or 'cloudsql-pg'"

# Input verification - Vector Store
assert vector_store in {'bigquery-vector', 'cloudsql-pgvector'}, "⚠️ Invalid VECTOR_STORE. Must be 'bigquery-vector' or 'cloudsql-pgvector'"

if logging: 
    assert bq_log_table_name, "⚠️ Please provide a name for your log table if you want to use logging"

if data_source == 'bigquery':
    assert bq_dataset_region, "⚠️ Please provide the Data Set Region"
    assert bq_dataset_name, "⚠️ Please provide the name of the dataset on Bigquery"

elif data_source == 'cloudsql-pg':
    assert pg_region, "⚠️ Please provide Region of the Cloud SQL Instance"
    assert pg_instance, "⚠️ Please provide the name of the Cloud SQL Instance"
    assert pg_database, "⚠️ Please provide the name of the PostgreSQL Database on the Cloud SQL Instance"
    assert pg_user, "⚠️ Please provide a username for the Cloud SQL Instance"
    assert pg_password, "⚠️ Please provide the Password for the PG_USER"


### 💾 **Save Configuration to File** 
Save the configurations set in this notebook to  `config.ini`. The parameters from this file are used in notebooks and in various modeules in the repo

In [6]:
from scripts import save_config

save_config(embedding_model, description_model, data_source, vector_store, logging, kgq_examples, PROJECT_ID,
            pg_region, pg_instance, pg_database, pg_user, pg_password, pg_schema, 
            bq_dataset_region, bq_dataset_name, 
            bq_opendataqna_dataset_name, bq_log_table_name, bq_table_list)

current dir:  /home/admin_/Talk2Data/notebooks
All configuration paramaters saved to file!


### ⚙️ **Database Setup for Vector Store: CloudSQL-pgvector**

Create PostgreSQL Instance on CloudSQL if 'cloudsql-pgvector' is chosen as vector store

Note that a PostgreSQL Instance on CloudSQL already exists if 'cloudsql-pg' is the data source. PostgreSQL Instance is created only if a different data store is chosen.

The cell will also create a dataset to store the log table on Big Query, **if** logging is enabled.

In [7]:
from env_setup import create_vector_store
# Setup vector store for embeddings
create_vector_store()  


current dir:  /home/admin_/Talk2Data/notebooks
root_dir set to: /home/admin_/Talk2Data
Initializing environment setup.
Loading configurations from config.ini file.
Data source set to:  bigquery
Logging is enabled
Vector store set to 'bigquery-vector'
Generating Big Query dataset opendataqna
Cannot find the dataset hence creating.......
ten-p-o.opendataqna is created


##  **1.2. Create Embeddings in Vector Store for RAG** 

### 🖋️ **Create Table and Column Embeddings**

In this step, table and column metadata is retreived from the data source and embeddings are generated for both

In [8]:
from env_setup import get_embeddings

# Generate embeddings for tables and columns
table_schema_embeddings, col_schema_embeddings = get_embeddings()  


Generating embeddings from source db schemas

LLM generated 1 Table Descriptions

LLM generated 7 Column Descriptions
Table and Column embeddings are created


### 💾 **Save the Table and Column Embeddings in the Vector Store**
The table and column embeddings created in the above step are save to the Vector Store chosen

In [9]:
from env_setup import store_embeddings

# Store table/column embeddings (asynchronous)
await(store_embeddings(table_schema_embeddings, col_schema_embeddings)) 


Storing embeddings back to the vector store.
Table and Column embeddings are saved to vector store


### 🗄️ **Load Known Good SQL into Vector Store**
Known Good Queries are used to create query cache for Few shot examples. Creating a query cache is highly recommended for best outcomes! 

The following cell will load the Natural Language Question and Known Good SQL pairs into our Vector Store. There pairs are loaded from `known_good_sql.csv` file inside scripts folder. If you have your own Question-SQL examples, curate them in .csv file before running the cell below. 

If no Known Good Queries are available at this time to create query cache, you can use [3_LoadKnownGoodSQL.ipynb](3_LoadKnownGoodSQL.ipynb) to load them later. 



#### Format of the Known Good SQL File (known_good_sql.csv)

prompt | sql | database_name [3 columns]

prompt ==> User Question 

sql ==> SQL for the user question (Note that the sql should enclosed in quotes and only in single line. Please remove the line  break)

database_name ==>This name should exactly  match the SCHEMA   NAME for Postgres Source or BQ_DATASET_NAME

In [10]:
from env_setup import create_kgq_table, store_kgq_embeddings

# Create table for known good queries (if enabled)
await(create_kgq_table()) 

# Store known good query embeddings (if enabled)
await(store_kgq_embeddings())  


⚠️ WARNING: No Known Good Queries are provided to create query cache for Few shot examples!
Creating a query cache is highly recommended for best outcomes
If no Known Good Queries for the dataset are availabe at this time, you can use 3_LoadKnownGoodSQL.ipynb to load them later!!
⚠️ WARNING: No Known Good Queries are provided to create query cache for Few shot examples!
Creating a query cache is highly recommended for best outcomes
If no Known Good Queries for the dataset are availabe at this time, you can use 3_LoadKnownGoodSQL.ipynb to load them later!!


### 🥁 If all the above steps are executed suucessfully, the following should be set up:

* GCP project and all the required IAM permissions

* Environment to run the solution

* Data source and Vector store for the solution

__________________________________________________________________________________________________________________